In [1]:
from PIL import Image

import csv
import glob
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torch.autograd as autograd         
from torch import Tensor                  
import torch.nn as nn                     
import torch.nn.functional as F           
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import captum

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class LevelClassifier(nn.Module):   
    def __init__(self):
        super(LevelClassifier, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 64, kernel_size=12, stride=4),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(64, 256, kernel_size=3, stride=2, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(256, 484, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(484),
            nn.ReLU(inplace=True),
            # Defining another 2D convolution layer
            nn.Conv2d(484, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(1536, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 32)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.linear_layers(x)
        return x

In [3]:
model = LevelClassifier()
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [4]:
guided_gc = captum.attr.GuidedGradCam(model, model.cnn_layers[-4])
input = torch.from_numpy(np.array(Image.open("../img/2-2_slice.png").convert('RGB')).astype(float).transpose(2, 0, 1)[np.newaxis, :, :, :]).float()
attribution = guided_gc.attribute(input, target=5)

AttributeError: module 'captum' has no attribute 'attr'

In [ ]:
model(input)